## AI, Decision-Making and Society Problem Set #1




***Problem Set 1 with adaptation for colab fix***
This template problem set version offers a workaround for google colab quota related issues. For section 3 - embeddings, instead of using the language model it loads the dataframe with embeddings from files (which you can also find on canvas, `train_df_embeddings.csv` and `test_df_embeddings.csv`). For problem 6, we provide the llm predictions based on example custom prompts. For full credit you should still create your own custom categories and prompts, but you can look at our categories, prompts and llm outputs `prompt_responses.csv` for your reflection.

You should be able copy appropriate code into your problem set fairly easily, simply replacing the block commented code with the new code in this template.

## Overview

The goal of this homework is to provide some hands-on experience of exploratory data analysis and data curation. We will work step-by-step through investigating a dataset, training some ML models, and evaluating the models. We will apply a minimal version of the Grounded Theory approach to define custom categories and consider the effects of different design decisions on model performance. Overall, the homework is divided into 3 parts:

1. Preparation: accessing the data and assessing the data source; preparing the environment and loading the data; preliminary exploration of the data.
2. Quantitative Methods: creating a numerical representation of the text data; training a simple classifier; evaluating model performance.
3. Qualitative Methods: exploring the grounded theory methods to look for qualitative or subjective patterns; open coding and focused coding; use of a large language model to augment exploration.


This notebook is designed to be run in a Google Colab environment.

Some problems or subproblems are marked as *graduate only*, meaning they are only required from students enrolled in the graduate version of the course (6.3952). Undergraduate students enrolled in the graduate version should complete these parts. Students enrolled in the undergraduate version of the course may still complete these problems but they will not be graded.

# Preparation

In this section, we will set up our environment for the rest of the assignment. We will:

*   Set up your colab to interact with the Gemini language model
*   Download a dataset to use for the assignment
*   Implement code to train a classifier



## Setting up your coding environment

First, we will set up the python environment to interact with libraries for model training and data manipulation.
Moreover, make sure to enable the GPU on the notebook by navigating to the 'runtime' tab, then clicking on 'change runtime type' and then selecting one of the available GPUs

In [ ]:
## Install the generative AI interface
!pip install -U -q google-generativeai
!pip install transformers

In [ ]:
## Imports in order to call relevant libraries
import re
import tqdm
import keras
import numpy as np
import pandas as pd
import os

import google.generativeai as genai

from google.colab import userdata

import seaborn as sns
import matplotlib.pyplot as plt

from keras import layers
from matplotlib.ticker import MaxNLocator
import sklearn.metrics as skmetrics

from transformers import pipeline
import torch
device = 0 if torch.cuda.is_available() else -1

Next, we will need to configure our code to connect to the language model server. You can do this with a Colab Secret named `GOOGLE_API_KEY`. If you don't already have this configured (or you're unsure if you do) follow the instructions in the [Authentication](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Authentication.ipynb) quickstart. You will find this easiest to do with a non-MIT google account.

In [ ]:
API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=API_KEY)

## Dataset

Now that we have configured our environment, we need some data to get started. For this assignment, we will use the [Employee Review dataset](https://www.kaggle.com/datasets/fiodarryzhykau/employee-review/code). This is a dataset of 880 employee performance reviews with a combined metric that asseses the performance & potential of the employees, measured on a scale from 1 to 9. We will consider the potential use of AI systems trained on this data.

Download the dataset from [Kaggle](https://www.kaggle.com/datasets/fiodarryzhykau/employee-review/code) and extract the csv files train_set.csv and test_set.csv to your local machine. Then, upload it to the Colab environment.

You can find instructions for uploading a file to a colab session [here](https://saturncloud.io/blog/how-to-use-google-colab-to-work-with-local-files/#:~:text=Uploading%20Files%20to%20Google%20Colab&text=Click%20on%20the%20%E2%80%9CFiles%E2%80%9D%20tab,for%20the%20upload%20to%20complete.).

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

## Question 1: Assessing the Data Source
Most data has documentation that describes how it was collected, what its intended purposes were, and known issues or risks. (If your data doesn't have this, its generally good practice to ask why!) Before we interact with the data, look at the documentation for this dataset and some example entries on [Kaggle](https://www.kaggle.com/datasets/fiodarryzhykau/employee-review/data). Answer the following questions with 1-3 sentences each.
<br><br>
**Related Reading**: [Datasheets for Datasets](https://arxiv.org/abs/1803.09010) articulates the motivation and reasoning behind this documentation.

### Q1.1: What are the risks and concerns associated with datasets containing employee reviews? Discuss if any of these risks can be mitigated and if so how.

The risks and concerns associated with datasets containing employee reviews that a LLM could potentially associate certain fake names with different performance categories. This risk can be mitigated by excluding the name feature in the review and only keep the unique identifier. Since knowing someone's name should not have an influence on the employee performance.

### Q1.2: What information is contained in this dataset? What is the purpose of this dataset?


The dataset contains a unique identifier to identify a fake employee, fake name, performance feedback and ranking, a category label to associate the feedback to performance level, whether the category was updated, and whether the fake employee feedback was thoroughly reviewed. The purpose of this dataset is to validate how good performance of deep learning models, that are potentially used for social science, are on raw data.

### Q1.3: How was this dataset collected?

This dataset was collected with the help of Amazon MTurk Workers. They were given a custom task to create fake employee reviews in order to make sure that there is a good level of variability and quality of data.

## Working with the Data

Now that we've considered the data source, let's look at the data! We'll do that by reading the .csv files into a [pandas](https://pandas.pydata.org/) dataframe (using the command pd.read_csv) and display the first few rows of the dataframe.

In [ ]:
train_df = pd.read_csv('train_set.csv')
train_df.set_index('id', inplace=True)

test_df = pd.read_csv('test_set.csv')
test_df.set_index('id', inplace=True)

labels_dict = dict(zip(test_df['nine_box_category'], test_df['label']))
labels_dict_reversed = dict(zip(test_df['label'], test_df['nine_box_category']))

Run this cell to see an example of what a data point from the training set looks like.

In [ ]:
train_df.head()

,person_name,nine_box_category,feedback,adjusted,reviewed,label,feedback_len,num_of_sent,performance_class,potential_class,feedback_clean,data_type
id,,,,,,,,,,,,
1,John Doe,"Category 1: 'Risk' (Low performance, Low poten...",John has not progressed in his position. He is...,False,True,0,287,5,0,0,john ha not progressed in his position he is c...,train
10045,Douglas Henry,"Category 1: 'Risk' (Low performance, Low poten...",Douglas Henry has been having trouble in all a...,False,False,0,430,6,0,0,douglas henry ha been having trouble in all ar...,train
10044,Douglas Henry,"Category 1: 'Risk' (Low performance, Low poten...",Douglas has a lot to work on and areas to grow...,False,False,0,290,4,0,0,douglas ha a lot to work on and area to grow i...,train
10005,Freddie Davies,"Category 1: 'Risk' (Low performance, Low poten...","Freddie is a nice guy, but his performance and...",False,True,0,418,5,0,0,freddie is a nice guy but his performance and ...,train
10004,Freddie Davies,"Category 1: 'Risk' (Low performance, Low poten...",Freddie has been quite disappointing this quar...,False,False,0,449,4,0,0,freddie ha been quite disappointing this quart...,train


For your convenience, we have provided a simple `multiline` function below to help reformat feedback text. This may assist you in later parts of the assignment, but you are not required to use it. The function attempts to render feedback with one sentence per line while ignoring periods appearing in initials and titles.

In [ ]:
multiline = lambda text: "\t"+re.sub(r'(\w{3})(\.+)(\s)', r'\1.\n\t',text)

## Example use
demo_feedback = train_df["feedback"][28]
print("Without multiline printing:")
print(demo_feedback)
print("\nWith multiline printing:")
print(multiline(demo_feedback))

Without multiline printing:
Mr. Hays is always running late or missing work. Mr. Hays has yet to hand in an assignment on time. Mr. Hays puts in little effort and work to his assignments. Doesn't appear to enjoy this job.

With multiline printing:
	Mr. Hays is always running late or missing work.
	Mr. Hays has yet to hand in an assignment on time.
	Mr. Hays puts in little effort and work to his assignments.
	Doesn't appear to enjoy this job.


## Question 2: Preliminary Exploration

In the follow section, answer questions about the training and test data. Below each question is a cell for code and a cell for text. The box of code should show your work, demonstrating the operations used to come up with your answer. You may feel free to add additional cells of code. At the end, include a text cell with your final answer and any discussion.


### Q2.1 Do labels correctly match performance and potential?

The dataset includes a column for the `nine_box_category` - a composite employee rating that combines information on their performance so far and their potential in the future. Each category in this system corresponds to a unique performance and potential rating, which can also be found in the columns `performance_class` and `potential_class` respectively. If, for example, an employee was marked as having `potential_class=2` and `performance_class=2`, but was given a label of 'Category 1: Risk', this would be an invalid entry. Does `df_train` contain any invalid entries due to incompatible `nine_box_category`,  `performance_class`, and `potential_class` values?

### NEED TO REDO THIS SECTION SO THAT IT CHECKS FOR THE PROPER PERFORMANCE CLASS AND POTENTIAL CLASS COMBINATIONS!!

In [ ]:
from enum import Enum

class ColumnName(Enum):
  NINE_BOX_CATEGORY = 'nine_box_category'
  INT_NINE_BOX_CATEGORY = 'int_nine_box_category'
  PERFORMANCE_CLASS = 'performance_class'
  POTENTIAL_CLASS = 'potential_class'
  REVIEWED = 'reviewed'
  ADJUSTED = 'adjusted'
  UPDATED = 'updated'

In [ ]:
print("Number of reviews: ", len(train_df))
pd.set_option('future.no_silent_downcasting', True)

Number of reviews:  656


In [ ]:
unique_categories = pd.unique(train_df[ColumnName.NINE_BOX_CATEGORY.value])
remapping_categories = {val:idx for idx, val in enumerate(unique_categories)}

train_df[ColumnName.INT_NINE_BOX_CATEGORY.value] = train_df[ColumnName.NINE_BOX_CATEGORY.value]
train_df[ColumnName.INT_NINE_BOX_CATEGORY.value] = train_df[ColumnName.INT_NINE_BOX_CATEGORY.value].replace(remapping_categories)

invalid_entries = train_df[train_df[ColumnName.INT_NINE_BOX_CATEGORY.value] != train_df[ColumnName.PERFORMANCE_CLASS.value]]
invalid_entries = invalid_entries[invalid_entries[ColumnName.INT_NINE_BOX_CATEGORY.value] != invalid_entries[ColumnName.POTENTIAL_CLASS.value]]
print("Number of invalid entries: ", len(invalid_entries))


Number of invalid entries:  417


In [ ]:
invalid_entries.head()

,person_name,nine_box_category,feedback,adjusted,reviewed,label,feedback_len,num_of_sent,performance_class,potential_class,feedback_clean,data_type,int_nine_box_category
id,,,,,,,,,,,,,
400,Joseph Burke,Category 4: 'Inconsistent Player' (Low perform...,Requires motivation to complete assignments. D...,True,True,3,153,4,0,1,requires motivation to complete assignment dev...,train,3
502,Kallie Beard,Category 4: 'Inconsistent Player' (Low perform...,Kallie Beard should show improvement in task p...,True,True,3,168,4,0,1,kallie beard should show improvement in task p...,train,3
558,Logan Ellis,Category 4: 'Inconsistent Player' (Low perform...,Logan Ellis is a average worker. I seen him de...,True,True,3,221,4,0,1,logan elli is a average worker i seen him dela...,train,3
551,Braiden Santos,Category 4: 'Inconsistent Player' (Low perform...,Braiden Santos has the capacity to do the work...,True,True,3,162,4,0,1,braiden santos ha the capacity to do the work ...,train,3
10014,Tom Houghton,Category 4: 'Inconsistent Player' (Low perform...,Tom Houghton has shown that his work consisten...,False,False,3,283,4,0,1,tom houghton ha shown that his work consistenc...,train,3


Yes, train_df contains 417 invalid entries due to incompatible nine_box_category, performance_class, and potential_class values.

### Q2.2 How well represented are each of the nine-box employee labels in the the training dataset?
How many training examples do we have for each performance review category 1-9? Which label is the least represented in the training data?

In [ ]:
train_df[ColumnName.NINE_BOX_CATEGORY.value].value_counts()

,count
nine_box_category,
"Category 1: 'Risk' (Low performance, Low potential)",97
"Category 5: 'Core Player' (Moderate performance, Moderate potential)",88
"Category 9: 'Star' (High performance, High potential)",87
"Category 4: 'Inconsistent Player' (Low performance, Moderate potential)",80
"Category 2: 'Average performer' (Moderate performance, Low potential)",72
"Category 3: 'Solid Performer' (High performance, Low potential)",70
"Category 6: 'High Performer' (High performance, Moderate potential)",66
"Category 8: 'High Potential' (Moderate performance, High potential)",65
"Category 7: 'Potential Gem' (Low performance, High potential)",31


In [ ]:
train_df[ColumnName.NINE_BOX_CATEGORY.value].value_counts(normalize=True) # equal proportion for each category
                                                                          # would mean each category has about 11%

,proportion
nine_box_category,
"Category 1: 'Risk' (Low performance, Low potential)",0.147866
"Category 5: 'Core Player' (Moderate performance, Moderate potential)",0.134146
"Category 9: 'Star' (High performance, High potential)",0.132622
"Category 4: 'Inconsistent Player' (Low performance, Moderate potential)",0.121951
"Category 2: 'Average performer' (Moderate performance, Low potential)",0.109756
"Category 3: 'Solid Performer' (High performance, Low potential)",0.106707
"Category 6: 'High Performer' (High performance, Moderate potential)",0.100610
"Category 8: 'High Potential' (Moderate performance, High potential)",0.099085
"Category 7: 'Potential Gem' (Low performance, High potential)",0.047256


Number of examples for each category:

*   Category 1: 'Risk' (Low performance, Low potential)	has 97 examples
*   Category 2: 'Average performer' (Moderate performance, Low potential)	has 72 examples
*   Category 3: 'Solid Performer' (High performance, Low potential) has	70 examples
*   Category 4: 'Inconsistent Player' (Low performance, Moderate potential)	has 80 examples
*   Category 5: 'Core Player' (Moderate performance, Moderate potential) has 88 examples
*   Category 6: 'High Performer' (High performance, Moderate potential)	has 66 examples
*   Category 7: 'Potential Gem' (Low performance, High potential)	has 31 examples
*   Category 8: 'High Potential' (Moderate performance, High potential)	has 65 examples
*   Category 9: 'Star' (High performance, High potential)	has 87 examples

And category 7 is least represented in the training data.

### Q2.3 What fraction of the training set and test set are verified?
This dataset is partially verified. What does that mean? What fraction of observations in the training data have been verified? What fraction of observations in the test data have been verified?

In [ ]:
train_df[ColumnName.REVIEWED.value].value_counts(normalize=True)

,proportion
reviewed,
False,0.710366
True,0.289634


In [ ]:
test_df[ColumnName.REVIEWED.value].value_counts(normalize=True)

,proportion
reviewed,
True,1.0


The dataset is partially verified, meaning that part of the dataset has been reviewed by a human to verify that the labels for category and employee performance are accurate and matching. About 28% of observations in the training data have been verified. 100% of observations in the testing data have been verified.

### Q2.4 What fractions of the training set and test set were updated after changed after verification?
What does this tell you about the data collection process? What implications does this have for developing our classification model?

In [ ]:
train_df[ColumnName.ADJUSTED.value].value_counts(normalize=True)

,proportion
adjusted,
False,0.859756
True,0.140244


In [ ]:
test_df[ColumnName.UPDATED.value].value_counts(normalize=True)

,proportion
updated,
True,0.546667
False,0.453333


This tells us that the data collection process has some issues since it relies on humans to verify the data. However, if humans are not able to verify all the data used then the model will be trained used erroneous data. This means that our classification model might not be accurate since the training data has contradictory category information.

### Q2.5 Display feedback matching the following criteria.

Create a filtered subset of the dataframe that contains only employees who had a label reviewed but not changed. From this filtered dataframe, display 5 feedback messages for Category 5 employees, and 5 feedback messages for Category 7 employees.

In [ ]:
print("Length of train_df: ", len(train_df))
train_df_reviewed = train_df[train_df[ColumnName.REVIEWED.value]]
print("Length of train_df_reviewed: ", len(train_df_reviewed))
train_df_review_not_adjusted = train_df_reviewed[~train_df_reviewed[ColumnName.ADJUSTED.value]]
print("length of train_df_review_not_adjusted: ", len(train_df_review_not_adjusted))
# train_df_review_not_adjusted.head()
filtered_category_5 = train_df_review_not_adjusted[train_df_review_not_adjusted[ColumnName.INT_NINE_BOX_CATEGORY.value] == 4][:5]
filtered_category_7 = train_df_review_not_adjusted[train_df_review_not_adjusted[ColumnName.INT_NINE_BOX_CATEGORY.value] == 6][:5]

Length of train_df:  656
Length of train_df_reviewed:  190
length of train_df_review_not_adjusted:  98


In [ ]:
print("Category 5 Feedback")
for review in filtered_category_5.feedback:
  print(multiline(review))
  print()

Category 5 Feedback
	Kieran is a valued team member.
	Kieran tries hard, but shows potential for further growth.
	Kieran could show more dedication to tasks at hand and focus to improve work performance.
	Kieran is a good fit for their current role in the company.

	Bobby's work is always flawless and I can trust that it's in good shape to send directly to a client without much review on my part.
	However, he pays such close attention to details that he's often unwilling to let go of a draft until he deems it perfect.
	This means he misses some key deadlines along the way.
	I've spoken with him about this issue and he's willing to focus on his time management as an area of development.
	However, he's such a perfectionist that I'm not sure whether he'll be able to make significant improvements in this area.

	Logan is a normal employee, his performance are not bad but not excellent.
	He is doing some sales and his number show us that he is not bad, He is always willing to take the harde

In [ ]:
print("Category 7 Feedback")
for review in filtered_category_7.feedback:
  print(multiline(review))
  print()

Category 7 Feedback
	Brantley has been with the firm for a year.
	He has proved himself to be intelligent, quick-learning, highly responsive.
	He is a great communicator.
	

Though due family issues, Brantley has not given his 100%. He has been lagging in delivering his work.
	He has been absent in meetings and occasional for work.
	Recommend that we allow him time to sort out his family issues and come back to contribute to the firm fully

	Brantley continues to be less productive than I believe he could be. He is intelligent and when he pushes himself, he has been able to suggest ways to increase productivity and profit in his department.
	If he improves in his productivity, he could become an invaluable asset to the company.
	He may need some guidance to find his ideal position.

	Brantley's time with our organization could be characterized as unusual in both good and bad ways.
	On one hand, he hasn't shown much in the way of numbers or standout work.
	On the other, he does show str

### Q2.6 (Graduate Only) Relationship between performance and potential

Create a table that gives the conditional probability $p(\verb|potential_class|=x|\verb|performance_class|=y)$.

Consider a new employee (not seen in the training or test data). Can the conditional probability table you create help you predict this new employee's potential if you know their performance rating?

In [ ]:
conditional_perf_pot = pd.crosstab(train_df[ColumnName.PERFORMANCE_CLASS.value], train_df[ColumnName.POTENTIAL_CLASS.value], normalize='index')
print(conditional_perf_pot)

potential_class           0         1         2
performance_class                              
0                  0.466346  0.384615  0.149038
1                  0.320000  0.391111  0.288889
2                  0.313901  0.295964  0.390135


Yes, you can use Bayesian inference to predict a performance rating given the employee's potential.

# Quantitative Methods

## Question 3: Text Embeddings


Below we have provided code to create embeddings (using the embedding-001 model). Run the following code and make sure that your colab notebook is enabled with the GPU configuration.

*Remark: the embedding generation might take a while, so be patient.*

In [ ]:

from tqdm.auto import tqdm
tqdm.pandas()

from google.api_core import retry

def make_embed_text_fn(model):

  @retry.Retry(timeout=300.0)
  def embed_fn(text: str) -> list[float]:
    # Set the task_type to CLASSIFICATION.
    embedding = genai.embed_content(model=model,
                                    content=text,
                                    task_type="classification")
    return embedding['embedding']

  return embed_fn

def create_embeddings(model, df):
  df['Embeddings'] = df['feedback'].progress_apply(make_embed_text_fn(model))
  return df

model_embeddings = 'models/embedding-001'
df_train = create_embeddings(model_embeddings, train_df)
df_test = create_embeddings(model_embeddings, test_df)


import ast
df_test=pd.read_csv('test_df_embeddings.csv', index_col=0)
df_train=pd.read_csv('train_df_embeddings.csv', index_col=0)
df_test['Embeddings'] = df_test['Embeddings'].apply(ast.literal_eval)
df_train['Embeddings'] = df_train['Embeddings'].apply(ast.literal_eval)

  0%|          | 0/656 [00:00<?, ?it/s]

RetryError: Timeout of 300.0s exceeded, last exception: 429 POST https://generativelanguage.googleapis.com/v1beta/models/embedding-001:embedContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.

### Q3.1 In your own words, describe what the code above is doing.

Creates a word embedding for the training and testing data, so that the qualitative feedback data is now stored as a quantative value. Then for each word embedding converts the literal into a python object.  

### Q3.2 What is the size of the embedding?

In [ ]:
# Enter  your code here


(Response for Q3.2)

### Q3.3 (Grad Students Only) Assessing Feedback Similarity

Once text is embedded and represented as a vector of floating point values, metrics allow us to quantify the similarity of two messages. In this problem, consider two groups of employees(indicated by their index in the training dataframe) that share feedback labels:

Group A: 10045, 10044, 10005

Group B: 702, 703, 704

Compute and present the cosine distance (`sklearn.metrics.pairwise::cosine_similarity`) between all pairs of individuals.

What might cosine distances be able to tell you about the appropriateness of this embedding for this classification task?

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
groupA =

(Response for Q3.3)

## Question 4: Model Training

If we look at the data in `df_train` and `df_test` we can see that there is a new column that includes the embeddings for the review. In the pandas dataframe, this column is stored as a Series of lists. Each list contains a numerical representation of that row's example according to a pre-trained model (we used `models/embeddings-01` from google ai). These lists do not have a ready interpretation to a human reader, but we will be able to use this numerical representation to train a custom classifer next.

### Q4.1 Creating Model Inputs

In order to use many common ML libraries, including those we will use in this assignment, we need to represent the covariates and target labels in an appropriate data type (eg `np.ndarray`). Complete the following function that takes in a column name, eg `label`, `performance_class`, or `potential_class` and returns as a tuple:
- a numpy array with the training data's covariates
- a number array with the training data's classification target
- a numpy array with the test data's covariates
- a number array with the test data's classification target


In [ ]:
def get_fit_data(target_column):
  covariate_columns_train = [clm_nm for col_nm in df_train if clm_nm != target_column]
  covariate_columns_test = [clm_nm for col_nm in df_test if clm_nm != target_column]

  x_train = df_train[covariate_columns_train].to_numpy()
  y_train = df_train[target_column].to_numpy()
  x_val = df_test[covariate_columns_test].to_numpy()
  y_val = df_test[target_column].to_numpy()

  return x_train, y_train, x_val, y_val

### Fitting the Model
We've implemented the following `build_classification_model` for you. This creates a a classifier comprised of two fully-connected layers, and will use it to classify the different performance/potential metric based on the feedback embeddings. It takes in `input_size` (the number of input features, which in this case is the size of the embedding) and `num_classes` which is the number of possible outputs of the classifier (note that `num_classes` will be different depending on what target column we intend to predict)


In [ ]:
def build_classification_model(input_size: int, num_classes: int) -> keras.Model:
  inputs = x = keras.Input(shape=(input_size,))
  x = layers.Dense(input_size, activation='relu')(x)
  x = layers.Dense(num_classes, activation='sigmoid')(x)
  classifier = keras.Model(inputs=[inputs], outputs=x)
  display(classifier.summary())
  classifier.compile(loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                   optimizer = keras.optimizers.Adam(learning_rate=0.0001),
                   metrics=['accuracy'])
  return classifier

The follow code can be used to visualize the training of the model:


In [ ]:
def plot_history(history):
  """
    Plotting training and validation learning curves.

    Args:
      history: model history with all the metric measures
  """
  fig, (ax1, ax2) = plt.subplots(1,2)
  fig.set_size_inches(20, 8)

  # Plot loss
  ax1.set_title('Loss')
  ax1.plot(history.history['loss'], label = 'train')
  ax1.plot(history.history['val_loss'], label = 'test')
  ax1.set_ylabel('Loss')

  ax1.set_xlabel('Epoch')
  ax1.legend(['Train', 'Validation'])

  # Plot accuracy
  ax2.set_title('Accuracy')
  ax2.plot(history.history['accuracy'],  label = 'train')
  ax2.plot(history.history['val_accuracy'], label = 'test')
  ax2.set_ylabel('Accuracy')
  ax2.set_xlabel('Epoch')
  ax2.legend(['Train', 'Validation'])

  plt.show()

### Q4.2 Implement the Training
Complete the code below that takes in a target column name (as a string, eg `"label"`) and completes the model training. You'll implement the lines containing "to-do" to. The rest of the function: builds the classifier using the function above, fits the model to the input data, evaluates the performance of the model on the held-out data, and - if the `show_plot=True` keyword argument is given - renders a plot to visualize the training history. The function returns the trained classifier, the history object, and a dictionary of evaluation results (as a tuple).

The number of epochs and batch size for training can also be modified via keyword arguments. Feel free to play with these values to see how it impacts training.

In [ ]:
## To be completed:
def train_classifier(target_column,  NUM_EPOCHS = 40, BATCH_SIZE = 8, show_plot = False):
  x_train, y_train, x_val, y_val = (None,None,None,None)## todo
  embedding_size = None # todo
  num_classes = None # todo

  classifier = build_classification_model(embedding_size, num_classes)
  history = classifier.fit(x=x_train,
                         y=y_train,
                         validation_data=(x_val, y_val),
                         batch_size=BATCH_SIZE,
                         epochs=NUM_EPOCHS,)
  evaluation = classifier.evaluate(x=x_val, y=y_val, return_dict=True)
  if show_plot:
    plot_history(history)
  return classifier, history, evaluation

### Q4.3 Train classifiers for `label`, `performance_class`, and `potential_class`

Using `train_classifier`, train classifiers for these three employee classifications.

What is the accuracy and loss for each classifier? How do these results compare? Why might this be?

Store the classifier and history outputs for the nine-box classification as global variables `classifier` and `history` respectively so they can be used in subequent problems. How you handle any other function outputs are up to you.

(Response for Q4.3)

### Q4.4 Intepreting model history
Run the code below (recall, `history` should be a global variable that contains the training history during training on the nine-box employee label). Then answer the following question:

Why is there a gap between the accuracy on the training set and the accuracy on the validation set? Why does the training loss decrease while the validation loss stagnates?

In [ ]:
plot_history(history)

(Response for Q4.4)

## Storing Predictions

The following code stores the predicted performance label from your classifier in the training and testing dataframes. Feel free to investigate these predictions to further explore the behavior of your model.

In [ ]:
x_train, y_train, x_val, y_val = get_fit_data("label")
y_hat_train = classifier.predict(x=x_train)
y_hat_train = np.argmax(y_hat_train, axis=1)
df_train['predicted_label'] = y_hat_train

y_hat_test = classifier.predict(x=x_val)
y_hat_test = np.argmax(y_hat_test, axis=1)
df_test['predicted_label'] = y_hat_test

# Qualitative Methods

## Question 5: Data Annotation

So far, over the course of the assignment, we have moved to interact more and more with the actual data:

1.   First, you looked at the data documentation and information about the dataset.
2.   Second, you looked an quantitative evaluation of a model using the existing labels.
3.   Third, you did a manual subjective evaluation of the model using your own interpretation of the labels.

Now, we'll go one step further and invent some new categories for the data. The first step is to do initial coding of the data. For this step, we will collect some free form annotations of the data.

*Hint*: If you're not sure how to go about annotating data, look at the content in Ch. 5 of Charmaz about initial coding.




#### Q5.1 Finish the following function to collect data annotations
This function should take in a dataframe `df` with a string-containing column "annotation" and an index `idx` for one of the rows in the input dataframe. The skeleton code added for you prints the feedback string, the true label, and the predicted label.

Your code should implement functionality to solicit input from the user and store the submitted annotation in the appropriate cell in the dataframe.

In [ ]:
df_train['annotation'] = ''
df_test['annotation'] = ''
def annotate(df, idx):
    row = df.loc[idx]
    print(f"Feedback is: {row['feedback']}")
    print(f"Original Label: {labels_dict_reversed[row['label']]}")
    print(f"Predicted Label: {labels_dict_reversed[row['predicted_label']]}")

    # Ask the user for an annotation for the example
    ## ENTER YOUR CODE HERE

    ## Update the dataframe to contain the anotation.
    ## ENTER YOUR CODE HERE

(Enter any written response for Q5.1 here, optional)

#### Q5.2 Implement Annotation Loop

Complete the `run_annotation_loop` below. The desired behavior will differ depending on the version of the course you are taking.

**For undergraduate version (6.3950)**: calling run annotation loop with parameter `examples_to_inspect` should prompt the user to annotate `examples_to_inspect` rows in the training dataframe and `examples_to_inspect` rows in the test dataframe. to_inspect>=15` for your annotating.

**For graduate version (6.3952)**: calling run annotation loop with parameter `examples_to_inspect` should prompt the user to annotate  `examples_to_inspect` rows in the training dataframe and `examples_to_inspect` rows in the test dataframe for each nine-box class label.

In [ ]:
def run_annotation_loop(examples_to_inspect):
  pass #ENTER YOUR CODE HERE

#### Q5.3 Preliminary annotations

Run `run_annotation_loop` and annotate the employee reviews you see. You are free to annotate as many entries as you want, and you may benefit from additional annotation in subsequent parts, but you should annotate at least 15 total entries. The corresponding minimum value of `examples_to_inspect` will depend on which version of the annotation loop you input above.

Then, produce a visual display of your annotations that you find suitable for searching for common themes and trends. This can be created using python or any other tools you find suitable. Include your code or a photo/screenshot to demonstrate your visual display below.

In [ ]:
n_examples = ###
run_annotation_loop(n_examples)


In [ ]:
## to-do: write code to visualize your annotations, or
## upload an image if your display is created using another software
## or by hand.

(Additional response area for Q5.3)

#### Q5.4 Focused Annotation

Now that we have our annotations, the next step is to move from initial coding to focused coding. Review your annotations and identify 3 interesting similarities or differences that you found in the data. Next propose 2 different ways of categorizing the data based on the similarities.

For example, your annotations may have noticed that some reviews focus on interpersonal behavior (e.g., how well the employee works on a team) while others focus on work quality (e.g., do they finish their work on time and in a quality manner). In this case, you might create a category called "Review Focus" with labels: "other", "interpersonal skills", "work quality".

For each category please:

1.   Provide a 1-2 sentence summary.
2.   List the labels in that category.
3.   Provide an example of each label.
4.   Describe why the categorization might be useful.

In [ ]:
## Create code that defines your categories
## Maps category name to (labels, summary, examples)

custom_categories = {'CUSTOM CATEGORY 1': (('LABEL 1', 'LABEL 2'),
                                           "CATEGORY 1 SUMMARY"),
                     'CUSTOM CATEGORY 2': (('LABEL 1', 'LABEL 2'),
                                           "CATEGORY 2 SUMMARY")}

## Create a dictionary that maps (Category, Label) -> Example
example_dict = {('CATEGORY', 'LABEL'): 'EXAMPLE FROM ANNOTATIONS'}


for category in custom_categories:
  print(f"Category is: {category}; {custom_categories[category][1]}")
  print(f"Labels are: {list(enumerate(custom_categories[category][0]))}")

In [ ]:
for category in custom_categories:
  df_train[category] = -1
  df_test[category] = -1

def custom_label(row, category, labels, summary):

  print(f"Feedback is: {row['feedback']}")
  print(f"Original Label: {labels_dict_reversed[row['label']]}")
  print(f"summary for category is: {summary}")

  # Ask the user for a label
  label = ## YOUR CODE HERE

  # Return the label
  return labels[int(label)]

# Apply the function to your dataset
examples_to_inspect = 20

def label_category(category, labels):

  print(f"Category is: {category}; {custom_categories[category][1]}")
  print(f"Labels are: {list(enumerate(labels))}")
  for label in labels:
    print(f"Example for {label} is: {example_dict[category, label]}")

  print('\n')
  print('=======================================================')
  print('\n')

  for i in range(len(df_train)):

    # Annotate an example from the training set
    df_train.iloc[i, df_train.columns.get_loc(category)]= custom_label(df_train.iloc[i],category,labels,custom_categories[category][1])

    # Annotate an example from the testing set
    df_test.iloc[i, df_test.columns.get_loc(category)]= custom_label(df_test.iloc[i],category,labels,custom_categories[category][1])

    if i >= examples_to_inspect:
        break

    # Print the current iteration index
    if i%3 == 0:
      print('=======================================================')
      print(f"=========Current i is {i}=========")

# Label each category
for category in custom_categories:
  print('=======================================================')
  label_category(category, labels = custom_categories[category][0])
  print('=======================================================')

[Q5.4 Prompt Reminder:
For each category please:
1.   Provide a 1-2 sentence summary.
2.   List the labels in that category.
3.   Provide an example of each label.
4.   Describe why the categorization might be useful.]

(Enter reflection on categories here)


#### Q5.5 Reflection
Write a 5-10 sentence reflection on the data annotationing process. What have you learned from categorizing data - either about the data or the process itself? How did the two-step process (of preliminary annotations and then focused annotations) help you learn about your data? Did you find it easier to annotate later examples compared to the first reviews you annotated? Explain why you chose to visualize your preliminary annotations the way you did and discuss whether this visualization was effective for inspiring your focused annotations or not. Did you encounter any challenges in this process?

Relate your reflection to what you learned from lectures and reading about grounded theory.

(Enter reflection for Q5.5 here)

## Question 6: (Graduate Only) Building a Model for Your Custom Labels

Now that we have some labels here, we will experiment with some methods for predicting your custom labels. One way to approach this would be to use the same method we did up above. If you'd like, you can implement that method and evaluate the new model quantiatively. However, you will likely find that performance is quite poor --- we tend to need more labels than is educationally productive to assign in a PSET!

An alternative is to turn our classification problem into a text completion task. In practice, this involves phrasing the classification task in natural language (i.e., English for our purposes here). For instance, if we wanted to classify whether sentence "I'm really disappointed that summer is over." is happy or sad, we could ask a language model to complete the following sentence:


```
"""
Is the following sentence happy?. Answer with "happy", "sad", or "neither". "I'm really disappointed that summer is over."
"""
```

While it isn't gauranteed that this will work, in practice these types of methods can work well with 0 additional data --- as long as they are evaluated properly. While this might seem like magic (and it is amazing), it's always a good idea to be skeptical of what seems like a free lunch. There are some important caveats that we need to consider:

*   The way you phrase your question directly determines the categories the model will recognize. Everything else will be influenced by the model's inherent biases. This isn't necessarily bad (for example, the model's bias towards generating coherent responses is crucial), but it means the model will rely on its data-driven interpretation of your prompt, which may or may not align with your intended meaning.

*  Small changes to prompts can change lead to large changes in behavior. [For example](https://arxiv.org/abs/2309.03409), including the phrase "Take a deep breath and work on this problem step-by-step" or offering to "pay" the model for better answers has improved performance in experiments. The process of iterating on these *prompts* has gotten the name *prompt engineering*. It's more of an iterative design exercise then a science, but [best practices](https://llama.meta.com/docs/how-to-guides/prompting/) are starting to emerge.

#### Optional Reading
While the idea of reducing one problem to another that you already know how to solve is one of the fundamentals in computer science, the application of this to large language models doing text completion tasks was introduced in "[Language Models are Unsupervised Multitask Learners.](https://paperswithcode.com/paper/language-models-are-unsupervised-multitask)"



#### Q6.1 Prompt Creation

Write out 3 potential prompts for each custom category you created. Explain your reasoning for the prompt in 1-3 sentences each. The prompts should be phrased as a question that the model can complete, aim to classify between the classes you created.

For example, if you had a category called "Work Quality" with labels "bad", "moderately bad", "moderately good" and "good worker", an example of a prompt can be: "Please classify the work quality of this employee according to the next list- bad, moderately bad, moderately good or good worker: {0}"
and where the {0} is replaced by the sentence you want to classify, which in our case is the feedback field.

In [ ]:
prompts = {'CUSTOM CATEGORY 1': ['YOUR PROMPT HERE',
                                 ## WHY MIGHT THIS PROMPT WORK?
                                 'YOUR PROMPT HERE',
                                 ## WHY MIGHT THIS PROMPT WORK?
                                 'YOUR PROMPT HERE',
                                 ## WHY MIGHT THIS PROMPT WORK?
                                 ],
           'CUSTOM CATEGORY 2': ['YOUR PROMPT HERE',
                                 ## WHY MIGHT THIS PROMPT WORK?
                                 'YOUR PROMPT HERE',
                                 ## WHY MIGHT THIS PROMPT WORK?
                                 'YOUR PROMPT HERE',
                                 ## WHY MIGHT THIS PROMPT WORK?
                                 ],}



#### Q6.2 Implement Custom Classification using a Language Model


Now that we have our prompts, we can evaluate them! We'll use the DistilBART language model, applying the designed prompts to the feedback field from the employees.


In [ ]:
# We will use the following model for this custom classification
language_model = pipeline("zero-shot-classification", model="valhalla/distilbart-mnli-12-1", device = device)



For example, consider an interaction with the model. Suppose the prompt you designed is: 'Does the following employee have good potential? Answer with "no", "mild potential", or "extraordinary potential",' and the feedback field is: 'John's performance was poor in the last quarter, although he graduated first in his class at MIT and scored an A+ in the AI, Decision-Making, and Society course.' In this case, we expect the model to return 'extraordinary potential'.

This following example templates how to use the `language_model`:

In [ ]:
input_example = "Do you think the following employee is liked or disliked by coworkers? \
Dan is described as being very helpful in the workplace. He likes to chat at lunch \
and organize office hangouts. Dan is reliable, on time, and humble."
labels = ['liked', 'disliked']
result = language_model(input_example, candidate_labels=labels)
result

Complete the `llm_predict_custom_classification` function below. You should use the `custom_categories` dictionary you created in Q6.4. We've provided skeleton code to assist you, but feel free to edit the structure if it accomplishes the same goal. Your goal is to use the prompts developed in 6.4 and the language model loaded above to create predictions for each custom class you specified. You should update the input dataframe to include columns that give the most likely class label, according to the language model, for each prompt associated with each custom category.

In [ ]:


# Helper function to format column names
# takes in a category name as a string and an integer prompt index
# and returns a column name in the form 'category_name_prompt_#'
def get_column_name(category_name, prompt_index):
  category_str = category_name.replace(' ', '_').lower()
  return f"{category_to_str(category_name)}_prompt_{prompt_index}"


# Function to classify feedback for specified category (specified as a string),
# and classifies the feedback in each row based on each prompt associated with the
# category. The function should update df to contain columns that contain the most
# likely prediction for the custom label based on the language model.

def llm_predict_custom_classification(df, category):
  labels = (None,None) ## todo: this should be a tuple of all possible classification labels for this custom category
  for prompt_idx in range(len(prompts[category])):
    prompt = None ## todo: the appropriate prompt template string from the custom prompts data structure
    column_name =  get_column_name(category_name, prompt_idx)
    df[column_name] = ""
    for idx, row in df.iterrows():
      description = " " ## todo: the feedback to be added to the prompt
      combined_input = " " ## todo: the full prompt to be given to the llm
      classification_result = None ## todo: use `language_model' to predict the custom classification. hint: look at the transformers.pipeline documentation
      df.loc[idx, column_name] = None ## todo: populate this cell with the most likely custom class label based on the language model


We will now run your function for each custom category from Q6.1 on a small subset of the training data (composed of the first 5 entries). The following cell will display the row id and each column produced by the above (that is, columns containing the substring "prompt"). You are free to add additional code cells here or in the following question to further investigate your results. You are also free to edit your prompts after seeing the predictions behave, but you should note this in the comment space below.

In [ ]:
'''
mini_df = df_train.head(5).copy()
for category in custom_categories:
  llm_predict_custom_classification(mini_df, category)
mini_df.filter(like="prompt")
'''

'''
Sample LLM predictions based on the following custom classes and prompts:
custom_categories = {'Work Quality': (('Bad', 'Good'),
                                           "Quality of work by the employee"),
                     'Personality type': (('Lazy', 'Nice person', 'Hard worker'),
                                           "Personality type, based on different characterizations from the feedback field")}

## Create a dictionary that maps (Category, Label) -> Example
example_dict = {('Work Quality'     , 'Bad'         ): 'Douglas is a very bad worker',
                ('Work Quality'     , 'Good'        ): 'Mary is a great employee',
                ('Personality type' , 'Lazy'        ): 'Freddie dont like to work and prefer being on vacations',
                ('Personality type' , 'Nice person' ): 'Ashley is pretty nice',
                ('Personality type' , 'Hard worker' ): 'Extremely dedicated works'}

prompts = {'Work Quality': ['Do you think the next worker is good or bad? {0}',
                            'What is your opinion of this worker? Please answer with good or bad: {0}',
                            'Please tell me if this is bad or good: {0}',
                            ],
           'Personality type': ['Based on this description, this person is lazy, nice or either hard worker? {0}',
                                'I would like to know if this person is lazy, nice or either hard worker? {0}',
                                'I think this person is lazy, but it can also be nice or either a hard worker. What do you think? {0}',
                                ],}
'''

mini_df = pd.read_csv("prompt_results.csv")

(Q6.2 comment space, optional)

#### Q6.3 Reflection
Read the employee feedback for the individuals included in the `mini_df` dataset and assess what you think the appropriate custom label for each of your custom classifications would be. How did the language model-based predictions compare to your assessment? How consistent were the language model outputs for each category? If some prompts consistently produced classifications that matched your hand-labeled classes more often, why do you think this happened?

In [ ]:
# Optional: enter any code used to explore results during evaluation and reflection

(Enter Q6.3 reflection here)